# Calculate the pH for monoprotic acid/base solution

In [27]:
solution ={'H+':[('HCl',0.5)],
           'OH-':[('NaOH',0.6)],
           'HA':[('AcOH',1.7378e-5 ,0.3)],
           'A-':[('NH3', 5.7544e-10,0.3)]}

from scipy.optimize import fsolve
import numpy as np
def dkp(solution, h):
    H = h
    OH = 10**(-14)/H
    positive = 0
    negative = OH - H
    for i in solution['H+']:
        negative += i[1]
    for i in solution['OH-']:
        positive += i[1]
    for name, ka, conc in solution['HA']:
        alpha = ka/(ka+H)
        negative += conc*alpha
    for name, ka, conc in solution['A-']:
        alpha = H/(H+ka)
        positive += conc*alpha
    h1 = negative - positive
    if h1 < 0:
        h1 = 1e-14/(-h1)
    return h1
def h_cal(solution, h_guess):
    def func(h):
        return dkp(solution, h)
    h_real=fsolve(func,h_guess)
    return h_real[0]

print('[H+] = ', h_cal(solution, 1e-9))
print('pH =', -np.log10(h_cal(solution, 1e-9)))


[H+] =  1.1506871238484794e-09
pH = 8.939042746620116


# Calculate the pH for all acid/base solution

In [30]:
#Format solution:
format_ = {'h_and_oh':[('C_H+ (1)','C_H+ (2)'),('C(OH-)(1)','C_OH- (2)')],
           'HA':['name','ka','C_HA','C_A-'],
            'H2A':[('name','ka1','ka2','C_H2A','C_HA-','C_A2-'),()],
           'H3A': [('name', 'ka1', 'ka2','ka3','C_H3A','C_H2A-','C_HA2-','C_A3-'),()],
           'H4A': [('name', 'ka1', 'ka2','ka3','ka4','C_H4A','C_H3A-','C_H2A2-','C_HA3-','C_A4-'),()]}
solution = {'h_and_oh':[(1,0.2),(0.1,0.5)],
            'HA':[],
            'H2A':[('CO2', 4.46683592e-7,4.6773514e-11, 0.2,0.1,0.4)],
           'H3A': [('H3PO4', 0.00707945784, 6.16595002e-8,4.7863009e-13,0.5,0,0.2,0.4)],
           'H4A': []}

from scipy.optimize import fsolve
import numpy as np
def dkp_multi(solution,h):
    H=h
    OH=1e-14/h
    positive=0
    negative=OH-h
    for i in solution['h_and_oh']:
        negative += sum(solution['h_and_oh'][0])
        positive += sum(solution['h_and_oh'][1])
    for name, ka1, ka2, c1, c2, c3 in solution['H2A']:
        c0 = c1+c2+c3
        alpha = (ka1*h+2*ka1*ka2)/(h**2+h*ka1+ka1*ka2)
        negative += c0*alpha - c2 - 2*c3
    for name, ka1, ka2, ka3, c1,c2,c3,c4 in solution['H3A']:
        c0 =c1+c2+c3+c4
        alpha = (ka1*h**2 + 2*h*ka1*ka2 + 3*ka1*ka2*ka3)/(h**3+h**2*ka1+h*ka1*ka2+ka1*ka2*ka3)
        negative += c0*alpha - c2 -2*c3 -3*c4
    if not solution['H4A'] is None:
        for name, ka1,ka2,ka3,ka4,c1,c2,c3,c4,c5 in solution['H4A']:
            c0 =c1+c2+c3+c4+c5
            alpha = (ka1*h**3 + 2*h**2*ka1*ka2 + 3*h*ka1*ka2*ka3 + 4*ka1*ka2*ka3*ka4)/(h**4+h**3*ka1+h**2*ka1*ka2+h*ka1*ka2*ka3+ka1*ka2*ka3*ka4)
            negative += c0*alpha - c2 -2*c3 -3*c4 -4*c5
    h1 = negative - positive
    if h1 < 0:
        h1 = 1e-14/(-h1)
    return h1
def h_cal(solution, h_guess):
    def func(h):
        return dkp_multi(solution, h)
    h_real=fsolve(func,h_guess)
    return h_real[0]
print('[H+] = ', h_cal(solution, 1e-12))
print('pH =', -np.log10(h_cal(solution, 1e-9)))

[H+] =  1.5353083571195963e-06
pH = 5.8138043860858035
